# Iteration 5: 

## Plan English Summary

In this iteration we add in some basic performance measure calculations

1. Number of cancelled elective operations.
2. Utilization of critical care beds.
3. The mean waiting time for a bed.

These calculations only use statistics collected in the results collection period.


## Notebook imports

In [ ]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [ ]:
display(Markdown("inputs/iteration_5.md"))

## Date and time of input to LLM

> 1540 30th November 2023 

## Observations on the LLM Output




## LLM Output 1:

In this modified code, the performance measures are calculated at the end of the simulation run, excluding any data collected during the warm-up period. The measures are then outputted in a pandas dataframe. The performance measures calculated are the number of elective operations cancelled, the percentage of the (number of beds multiplied by the results collection period) that beds were in use by a patient, and the mean waiting time for a bed.

In [1]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)
        self.elective_cancelled = 0
        self.total_bed_usage = 0
        self.unplanned_wait_times = []

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
                self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
                self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
                self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
                self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
                self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            if self.env.now > self.experiment.warm_up_period:
                self.patient_count += 1
                if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                    self.elective_cancelled += 1
                    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                else:
                    print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
                    self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            self.total_bed_usage += treatment_time
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            self.total_bed_usage += treatment_time
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)

        unplanned_wait_times = [t for t in self.unplanned_wait_times if t > 0]
        mean_unplanned_wait_time = np.mean(unplanned_wait_times) if unplanned_wait_times else 0

        results = {
            'Elective Operations Cancelled': self.elective_cancelled,
            'Total Bed Usage (%)': (self.total_bed_usage / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)) * 100,
            'Mean Unplanned Admission Wait Time': mean_unplanned_wait_time
        }

        results_df = pd.DataFrame(results, index=[0])
        print(results_df)

In [2]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from Accident and Emergency at time 729.5429274168578
Patient 1 admitted to critical care bed at time 729.5429274168578
Patient 2 arrived from Accident and Emergency at time 731.0991420221671
Patient 2 admitted to critical care bed at time 731.0991420221671
Elective surgery patient 3 arrived at time 733.0003727840715
Elective surgery patient 3 admitted to critical care bed at time 733.0003727840715
Elective surgery patient 3 discharged from critical care bed at time 736.1600102010805
Critical care bed is available for next patient at time 741.1600102010805
Patient 4 arrived from the Wards at time 741.8454903787165
Patient 4 admitted to critical care bed at time 741.8454903787165
Patient 5 arrived from Accident and Emergency at time 752.57738997893
Patient 5 admitted to critical care bed at time 752.57738997893
Patient 5 discharged from critical care bed at time 752.6009559359751
Elective surgery patient 6 arrived at time 753.804238491233
Elective surgery patient 6 adm

## Modifications to LLM code to test for correct model outputs.

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [ ]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [ ]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [ ]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [ ]:
extreme_value_test_1()

### Test 2: Extreme value test 2

PASS

In [ ]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [ ]:
extreme_value_test_2()

### Test 3: Different run length

PASS

In [ ]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [ ]:
test_results_collection_period(10)

### Test 4: Extreme value test 3

PASS.

In [ ]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [ ]:
extreme_value_test_3()

### Test 5: Extreme value test 4

PASS

In [ ]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [ ]:
extreme_value_test_4()

### Test 6: Vary Warm-up parameter

PASS

In [ ]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [ ]:
test_warmup_period(0)